In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('data/train_cleaned.csv')

train_data.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,1.0,0.0,530001,38.1,132.0,24.0,1.0,4.0,2.0,2.0,...,57.0,8.5,3.0,3.4,1.0,2209,0,0,0.0,0.0
1,1.0,0.0,533836,37.5,88.0,12.0,1.0,3.0,5.0,2.0,...,33.0,64.0,3.0,2.0,1.0,2208,0,0,0.0,1.0
2,1.0,0.0,529812,38.3,120.0,28.0,1.0,4.0,6.0,1.0,...,37.0,6.4,3.0,3.4,1.0,5124,0,0,0.0,2.0
3,1.0,0.0,5262541,37.1,72.0,30.0,0.0,4.0,6.0,2.0,...,53.0,7.0,1.0,3.9,1.0,2208,0,0,1.0,2.0
4,0.0,0.0,5299629,38.0,52.0,48.0,3.0,3.0,4.0,1.0,...,47.0,7.3,1.0,2.6,0.0,0,0,0,1.0,2.0


opdel data i features og labels

In [3]:
X = train_data.drop(columns=['outcome'])
y = train_data['outcome']

X_train, X_test,\
    y_train, y_test = train_test_split(X, y,
                                       test_size=0.2,
                                       random_state=42)

In [4]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Data scaler
    ('clf', LogisticRegression())  # Logistisk regression model
])

param_grid = {
    'C': np.logspace(-5, 5, 11),  # Regularization strength
    'penalty': ['l1', 'l2'],  # Type of regularization
    'solver': ['liblinear', 'saga']  # Solver for optimization
}


In [5]:
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision_macro',
    'recall': 'recall_macro',
    'f1': 'f1_macro',
    'roc_auc': 'roc_auc_ovr'  # Use roc_auc_ovr for multiclass classification
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring=scoring, refit='accuracy')  # cv er antallet af krydsvalideringsfold

grid_search.fit(X_train, y_train)

c:\Users\Christian\anaconda3\envs\ml-env\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Christian\anaconda3\envs\ml-env\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Christian\anaconda3\envs\ml-env\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Christian\anaconda3\envs\ml-env\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']},
             refit='accuracy',
             scoring={'accuracy': 'accuracy', 'f1': 'f1_macro',
                      'precision': 'precision_macro', 'recall': 'recall_macro',
                      'roc_auc': 'roc_auc_ovr'})

In [6]:
# Get the best model and evaluate on the test data
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

# Evaluate multiple metrics on the test data
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
roc_auc = roc_auc_score(y_test, best_model.predict_proba(X_test), average='macro', multi_class='ovr')

# Print the best hyperparameters and test metrics
print("Best Hyperparameters:", grid_search.best_params_)
print("Test Metrics:")
print("  Accuracy:", accuracy)
print("  Precision:", precision)
print("  Recall:", recall)
print("  F1 Score:", f1)
print("  ROC AUC Score:", roc_auc)

Best Hyperparameters: {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Test Metrics:
  Accuracy: 0.6680161943319838
  Precision: 0.6630436732131647
  Recall: 0.6504948474645444
  F1 Score: 0.6537708378712562
  ROC AUC Score: 0.8252039724492229
